In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%cd ./drive/MyDrive/Colab\ Notebooks/NLP_Project/

/content/drive/.shortcut-targets-by-id/1zPjf1cHfdKqObemkPReffGbQHU_wotr2/NLP_Project


In [12]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
import random as rnd
from torch.optim.lr_scheduler import StepLR
import gc

In [ ]:
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

In [ ]:
# import pandas as pd

# # Replace these file paths with the actual paths to your CSV files
# file1_path = './Results/test_set_without_labels.csv'
# file2_path = './Results/result_compare_model_baseline.csv'

# # Load CSV files into DataFrames
# df1 = pd.read_csv(file1_path)
# df2 = pd.read_csv(file2_path)

# # Check if "letter" column exists in both DataFrames
# if 'letter' not in df1.columns:
#     print('Error: "letter" column not found in the first CSV file.')
#     exit()

# if 'input' not in df2.columns:
#     print('Error: "input" column not found in the second CSV file.')
#     exit()
# C = 0
# # Compare "letter" column in the first CSV with "input" column in the second CSV using a for loop
# mismatched_indices = []
# for idx, letter_value in enumerate(df1['letter']):
#     # if df2['input'][idx] == df2['input'][13994]:
#     #   C+=1
#     if df1['letter'][idx] != df2['input'][idx]:
#       print("err in index :", idx," & line : ", df1['line_number'][idx])
#       print("'",df1['letter'][idx - 4] ,"' --- '", df2['input'][idx - 4],"'")
#       print("'",df1['letter'][idx - 3] ,"' --- '", df2['input'][idx - 3],"'")
#       print("'",df1['letter'][idx - 2] ,"' --- '", df2['input'][idx - 2],"'")
#       print("'",df1['letter'][idx - 1] ,"' --- '", df2['input'][idx - 1],"'")
#       print("'",df1['letter'][idx] ,"' --- '", df2['input'][idx],"'")
#       print("'",df1['letter'][idx + 1] ,"' --- '", df2['input'][idx + 1],"'")
#       print("'",df1['letter'][idx + 2] ,"' --- '", df2['input'][idx + 2],"'")
#       print("'",df1['letter'][idx + 3] ,"' --- '", df2['input'][idx + 3],"'")
#       print("'",df1['letter'][idx + 4] ,"' --- '", df2['input'][idx + 4],"'")
#       print("'",df1['letter'][idx + 5] ,"' --- '", df2['input'][idx + 5],"'")
#       print("'",df1['letter'][idx + 6] ,"' --- '", df2['input'][idx + 6],"'")
#       print("'",df1['letter'][idx + 7] ,"' --- '", df2['input'][idx + 7],"'")
#       print("'",df1['letter'][idx + 8] ,"' --- '", df2['input'][idx + 8],"'")
#       break
#     # for idx_in, input_value in enumerate(df1['input']):

#       # if letter_value not in df2['input'].values:
#       #     mismatched_indices.append(idx)


# print(C)

# Constants

In [4]:
MODEL = "BI_LSTM"
NUM_LAYERS = 2
EMBEDDING_SIZE = 300

In [5]:
def get_vocab(vocab_path, tags_path):
    vocab = {}
    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i  # to avoid the 0
        # loading tags (we require this to map tags to their indices)
    vocab['<PAD>'] = len(vocab) # 35180
    tag_map = {}
    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i

    return vocab, tag_map

def get_params(vocab, tag_map, sentences_file, labels_file):
    sentences = []
    labels = []

    with open(sentences_file) as f:
        for sentence in f.read().splitlines():
            # replace each token by its index if it is in vocab
            # else use index of UNK_WORD
            s = [vocab[token] if token in vocab
                 else vocab['UNK']
                 for token in sentence.split(' ')]
            sentences.append(s)

    with open(labels_file) as f:
        for sentence in f.read().splitlines():
            # replace each label by its index
            s = sentence.split(' ')
            # remove empty strings
            s = list(filter(None, s))
            l = [tag_map[label] for label in s] # I added plus 1 here
            labels.append(l)
    return sentences, labels, len(sentences)


# Importing and discovering the data

In [6]:
vocab, tag_map = get_vocab('./Dataset/new_new_characters/unique_chars.txt', './Dataset/new_new_characters/unique_labels.txt')
t_sentences, t_labels, t_size = get_params(vocab, tag_map, './Dataset/new_new_characters/t_chars.txt', './Dataset/new_new_characters/t_labels.txt')
v_sentences, v_labels, v_size = get_params(vocab, tag_map, './Dataset/new_new_characters/v_chars.txt', './Dataset/new_new_characters/v_labels.txt')
test_sentences, test_labels, test_size = get_params(vocab, tag_map, './Dataset/new_new_characters/test_chars.txt', './Dataset/new_new_characters/test_labels.txt')

In [ ]:
# # test_sentences, test_labels, test_size = get_params(vocab, tag_map, './Dataset/new_new_characters/test_no_diacritics_chars.txt', './Dataset/new_new_characters/test_no_diacritics_labels.txt')
# test_sentences, test_labels, test_size = get_params(vocab, tag_map, './Dataset/new_new_characters/test2_chars.txt', './Dataset/new_new_characters/test2_labels.txt')

In [ ]:

# NOTE: to increase the size of the dataset
# t_sentences  = t_sentences + v_sentences
# t_labels = t_labels + v_labels
# t_size = t_size + v_size

In [7]:
# # vocab translates from a word to a unique number
# print('vocab["الأعم"]:', vocab["الأعم"])
# Pad token
print('padded token:', vocab["<PAD>"])

padded token: 38


In [8]:
# The possible tags
print(tag_map)

{'FATHA': 0, 'FATHATAN': 1, 'DAMMA': 2, 'DAMMATAN': 3, 'KASRA': 4, 'KASRATAN': 5, 'SUKUN': 6, 'SHADDA': 7, 'SHADDA_FATHA': 8, 'SHADDA_FATHATAN': 9, 'SHADDA_DAMMA': 10, 'SHADDA_DAMMATAN': 11, 'SHADDA_KASRA': 12, 'SHADDA_KASRATAN': 13, '_': 14, 'pad': 15}


In [ ]:

# # Exploring information about the data
# print('The number of outputs is tag_map', len(tag_map))
# # The number of vocabulary tokens (including <PAD>)
# g_vocab_size = len(vocab)
# print(f"Num of vocabulary words: {g_vocab_size}")
# print('The vocab size is', len(vocab))
# print('The training size is', t_size)
# print('The validation size is', v_size)
# print('An example of the first sentence is', t_sentences[0])
# print('An example of its corresponding label is', t_labels[0])
# len(t_sentences[0])==len( t_labels[0])

# NERDataset
The class that impelements the dataset for NER

In [9]:
class NERDataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad):
    self.x = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in x], padding_value=pad,batch_first = True)
    self.y = nn.utils.rnn.pad_sequence([torch.tensor(i) for i in y], padding_value=tag_map["pad"],batch_first = True)
    print('The max length of the sentences is', self.x.shape[1])
    print('The max length of the labels is', self.y.shape[1])
  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

# Classifiers
The class that implementss the pytorch model for arabic diacritic classification

In [ ]:
print(vocab)

{'ل': 0, 'ن': 1, 'ش': 2, 'آ': 3, 'ك': 4, 'ب': 5, 'ح': 6, 'ص': 7, 'ئ': 8, 'خ': 9, 'ي': 10, 'ز': 11, 'ض': 12, 'ط': 13, 'ج': 14, 'ر': 15, 'ة': 16, 'و': 17, 'ا': 18, 'غ': 19, 'ى': 20, 'ء': 21, 'ظ': 22, 'س': 23, 'ق': 24, 'د': 25, 'ف': 26, 'إ': 27, 'ذ': 28, 'ت': 29, 'ث': 30, 'م': 31, 'ع': 32, 'ه': 33, 'أ': 34, 'ؤ': 35, 'UNK': 36, '<pad>': 37, '<PAD>': 38}


In [ ]:
# # GRU

# class ArabicDiacriticsClassifier(nn.Module):
#     def __init__(self, vocab_size=len(t_sentences) + len(test_sentences) + len(v_sentences), num_layers=2, embedding_dim=300, hidden_size=50, n_classes=len(tag_map)):
#         MODEL = "BI_GRU"
#         super(ArabicDiacriticsClassifier, self).__init__()
#         # (1) Create the embedding layer
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)

#         # (2) Create a GRU layer with hidden size = hidden_size and batch_first = True
#         self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True, bidirectional=True)
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size

#         # (3) Create a linear layer with the number of neurons = n_classes
#         self.linear = nn.Linear(2 * hidden_size, n_classes)

#     def forward(self, sentences):
#         embeddings = self.embedding(sentences)

#         # Initialize hidden states for bidirectional GRU
#         h0 = torch.zeros(self.num_layers*2, embeddings.size(0), self.hidden_size).to(sentences.device)
#         gru_out, _ = self.gru(embeddings, h0)

#         final_output = self.linear(gru_out)
#         return final_output

In [18]:
class ArabicDiacriticsClassifier(nn.Module):
  def __init__(self, vocab_size=len(t_sentences) + len(v_sentences)+len(v_sentences), num_layers = 3, embedding_dim = 512, hidden_size=256, n_classes=len(tag_map)):
    super(ArabicDiacriticsClassifier, self).__init__()
    # (1) Create the embedding layer
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    # import gensim.downloader as api
    # self.embedding.weight.data.copy_(torch.from_numpy(api.load('word2vec-google-news-300').vectors[:1000])) WORD2VEc
    # import fasttext.util
    # self.embedding.weight.data.copy_(torch.from_numpy(fasttext.util.download_model('en', if_exists='ignore').get_input_matrix()[:1000]))

    # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
    self.num_layers = num_layers
    self.hidden_size = hidden_size


    self.lstm = nn.LSTM(embedding_dim, hidden_size,num_layers, batch_first=True, bidirectional=True)
    self.linear = nn.Linear(2 * hidden_size, n_classes)

    # (3) Create a linear layer with number of neorons = n_classes
    # self.linear = nn.Linear(hidden_size, n_classes)


  def forward(self, sentences):
    embeddings = self.embedding(sentences)

    # BIDIRECTIONAL
    # Initialize hidden states for bidirectional LSTM
    # h0 = torch.zeros(self.num_layers*2, embeddings.size(0), self.hidden_size).to(sentences.device)
    # c0 = torch.zeros(self.num_layers*2, embeddings.size(0), self.hidden_size).to(sentences.device)
    # lstm_out, (a, b) = self.lstm(embeddings, (h0, c0))

    # LSTM
    lstm1_out, (a,b) = self.lstm(embeddings)
    final_output = self.linear(lstm1_out)
    # final_output = self.linear(lstm_out[:, -1, :])
    return final_output

In [19]:
model = ArabicDiacriticsClassifier()
print(model)

ArabicDiacriticsClassifier(
  (embedding): Embedding(128137, 512)
  (lstm): LSTM(512, 256, num_layers=3, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=16, bias=True)
)


# Training

In [13]:
def train(model, train_dataset,start_epoch=0, batch_size=64, epochs=10, learning_rate=0.001):
  # (1) create the dataloader of the training set (make the shuffle=True)
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # (2) make the criterion cross entropy loss
  criterion = torch.nn.CrossEntropyLoss(ignore_index=tag_map["pad"])

  # (3) create the optimizer (Adam)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # define your scheduler
  # scheduler is used to decrease the learning rate by a factor of [gamma] every [step_size] epochs
  scheduler = StepLR(optimizer, step_size=1, gamma=0.00001)
  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    torch.cuda.empty_cache()
    criterion = criterion.cuda()
    torch.cuda.empty_cache()

  print("Resuming from epoch ", start_epoc)

  for epoch_num in range(start_epoch,epochs):
    total_acc_train = 0
    total_loss_train = 0
    prev_acc = -1
    for train_input, train_label in tqdm(train_dataloader):

      # (4) move the train input to the device
      train_label = train_label.to(device)

      # (5) move the train label to the device
      train_input = train_input.to(device)


      # (6) do the forward pass
      output = model(train_input)
      # print("output.shape",output.shape)
      # print("train_label.shape",train_label.shape)
      # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
      batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

      # (8) append the batch loss to the total_loss_train
      total_loss_train += batch_loss

      # (9) calculate the batch accuracy (just add the number of correct predictions)
      acc = (output.argmax(2) == train_label).sum().item()

      total_acc_train += acc

      # (10) zero your gradients
      optimizer.zero_grad()


      # (11) do the backward pass
      batch_loss.backward()


      # (12) update the weights with your optimizer
      optimizer.step()

      # update the learning rate
      scheduler.step()

    # epoch loss
    epoch_loss = total_loss_train / len(train_dataset)

    # (13) calculate the accuracy
    epoch_acc = total_acc_train / (len(train_dataset) * train_dataset[0][0].shape[0])
    if prev_acc >= epoch_acc:
      print("Finish training because too many epochs")
      return
    else:
      prev_acc = epoch_acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')
    model_name = f"model_baseline"

    torch.save(model.state_dict(), f'./BaseLineModels/{model_name}_EPOCH{epoch_num + 1}')

    # clear the cache each epoch

  ##############################################################################################################

In [14]:
train_dataset = NERDataset(t_sentences, t_labels, vocab['<PAD>'])
# val_dataset = NERDataset(v_sentences, v_labels, vocab['<PAD>'])
test_dataset = NERDataset(test_sentences, test_labels, vocab['<PAD>'])

The max length of the sentences is 1815
The max length of the labels is 1815
The max length of the sentences is 1936
The max length of the labels is 1936


In [15]:
def load_model(model,model_name):
  model.load_state_dict(torch.load(f'./SavedModels/{model_name}'))
  return model

def load_baseline_model(model,model_name,epoch_num):
  model.load_state_dict(torch.load(f'./BaseLineModels/{model_name}_EPOCH{epoch_num}'))
  return model

In [ ]:
model_name = f"model_baseline"
start_epoc = 5
if start_epoc != 0:
  model = load_baseline_model(model, model_name, start_epoc)


train(model, train_dataset,start_epoch = start_epoc, epochs = 10)
torch.save(model.state_dict(), f'./SavedModels/{model_name}')
# model = load_model(model, model_name)

Resuming from epoch  5


100%|██████████| 1818/1818 [39:38<00:00,  1.31s/it]


Epochs: 6 | Train Loss: 0.001386831165291369         | Train Accuracy: 0.9951628988617687



 99%|█████████▉| 1803/1818 [39:20<00:19,  1.31s/it]

In [ ]:
del train_dataset
# del val_dataset
gc.collect()

# Evaluation

In [ ]:
diacritic_results = []
gold_results = []
test_input_list = []


In [ ]:
def evaluate(model, test_dataset, batch_size=64):
  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  total_acc_test = 0


  # (2) disable gradients
  with torch.no_grad():

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)

      # (5) do the forward pass
      output = model(test_input)
      prediction = output.argmax(2)


      diacritic_results.extend(np.array(prediction.cpu().data).flatten())
      gold_results.extend(np.array(test_label.cpu().data).flatten())
      test_input_list.extend(np.array(test_input.cpu().data).flatten())

      # for i in range(test_input.shape[0]):
      #   for j in range(test_input.shape[1]):
      #     # result['id'].append(count)
      #     # count += 1
      #     result.append(prediction[i][j].cpu().data.numpy()[0])


      # accuracy calculation (just add the correct predicted items to total_acc_test)
      # acc = (prediction == test_label).sum().item()
      # total_acc_test += acc

      # if total_acc_test ==0:
      #   print("test_input",test_input.shape)
      #   print("test_label",test_label.shape)
      #   print("output",output.shape)
      #   print("output.argmax(2)",output.argmax(2).shape)
      # del output
      # del prediction

    # (6) calculate the over all accuracy
    # total_acc_test /= (len(test_dataset) * test_dataset[0][0].shape[0])
  ##################################################################################################


  # print(f'\nTest Accuracy: {total_acc_test}')

In [ ]:
# Give the model a name
# model_name = f"model_{MODEL}_{NUM_LAYERS}Layers_chars_only"
model_name = f"model_baseline"

### To Load an Existing Model

In [ ]:
# device = torch.device('cuda')
# model.load_state_dict(torch.load(f'./SavedModels/{model_name}'))
# model = model.to(device)

### Evaluatio and Save

In [ ]:
evaluate(model, test_dataset)

In [ ]:
# inputs = [LIST_OF_ARABIC_LETTERS[test_input_list[i]] for i in range(10)]
# print(inputs)

In [ ]:
gc.collect()

In [ ]:
der = 0
total_size = 0
for i in range(len(diacritic_results)):
  if test_input_list[i] != vocab['<PAD>']: # Do not include padding in DER calculations
    if diacritic_results[i] != gold_results[i] : # Miss Classification
      der += 1
    total_size += 1
der /= total_size
der *= 100
print("DER = ",der,"%")
print("Accuracy = ",100 - der,"%")

In [ ]:
# Save the Diacritic Classifier Model

In [ ]:
# these list are sorted as mentioned by the TA
LIST_OF_DIACRITICS = [
    "FATHA",
    "FATHATAN",
    "DAMMA",
    "DAMMATAN",
    "KASRA",
    "KASRATAN",
    "SUKUN",
    "SHADDA",
    "SHADDA_FATHA",
    "SHADDA_FATHATAN",
    "SHADDA_DAMMA",
    "SHADDA_DAMMATAN",
    "SHADDA_KASRA",
    "SHADDA_KASRATAN",
    "_"
]
LIST_OF_ARABIC_LETTERS = list(vocab.keys())
print(LIST_OF_ARABIC_LETTERS)
s = ['آ' ,'ض','ف','ص','أ','ت','ق','ث','ا','ه','غ','ة','ج','ك','م','ن','ي','ب','د','س','و','ل','ؤ','ش','إ','ط','ئ','ظ','ز','ى','ء','ر','ع','ذ','ح','خ','UNK','<pad>']
# for i in range(len(s)):
#   if s[i] not in LIST_OF_ARABIC_LETTERS:
#     print("####sd")

In [ ]:
print(tag_map)

In [ ]:
# Prepare the data that will be written in the CSV file
index = 5000
filtered_diacritic_results = [] # diacrtic results without paddings
filtered_gold_results = [] # diacrtic gold results without paddings
filtered_inputs = [] # inputs without paddings
state = [] # This marks the incorrect results in the CSV file

for i in range(len(diacritic_results)):
  if test_input_list[i] != vocab['<PAD>']:
    filtered_diacritic_results.append(diacritic_results[i])
    # filtered_gold_results.append(gold_results[i])
    filtered_inputs.append(test_input_list[i])
    # if diacritic_results[i] != gold_results[i]:
    #   state.append("Incorrect")
    # else:
    #   state.append("")

index = len(filtered_diacritic_results)
print(len(filtered_diacritic_results))
# print(len(filtered_gold_results))
print(len(filtered_inputs))

inputs = [LIST_OF_ARABIC_LETTERS[filtered_inputs[i]] for i in range(index)]
model_prediction = [LIST_OF_DIACRITICS[filtered_diacritic_results[i]] for i in range(index)]
# gold_out = [LIST_OF_DIACRITICS[filtered_gold_results[i]] for i in range(index)]
# state = state[0:index]
print(inputs[0:100])

In [ ]:
# Output the accuracy to CSV file
df = pd.DataFrame(
    {
     'ID': range(len(filtered_diacritic_results[0:index])),
     'label': filtered_diacritic_results[0:index],
     # The following columns are provided for illustrative purposes only and are not necessary (will be commented when submitting the CSV file).
     'input':  inputs,
    #  'diactric':  model_prediction,
    #  'gold out': gold_out,
    #  'state' : state
     })

df.to_csv(f'./Results/result_compare_{model_name}.csv', index=False)
# df.to_csv(f'./Results/result_{model_name}.csv', index=False)